# Profile HMMs for Sequence Alignment

## 1. Probability of a Hidden Path Problem: Compute the probability of a hidden path.

Input: A hidden path π followed by the states States and transition matrix Transition of an HMM (Σ, States, Transition, Emission).

Output: The probability of this path, Pr(π).

Note: You may assume that transitions from the initial state occur with equal probability.

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
def hpp(text):
    text = text.split('--------')
    string = text[0][:-1]
    
    matrix = pd.DataFrame(index=text[1][1:-1].split(' '),
                          columns=text[1][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[2])]).reshape((2,2)))
    prob = 0.5
    for i in range(len(string)-1):
        prob*=matrix[string[i]][string[i+1]]
    return prob

In [3]:
text = '''ABABBBAAAA
--------
A B
--------
	A	B
A	0.377	0.623
B	0.26	0.74'''

In [4]:
hpp(text)

0.0003849286917546759

## 2. Probability of an Outcome Given a Hidden Path Problem: Compute the probability that an HMM will emit a string given its hidden path.

Input: A string x, followed by the alphabet from which x was constructed, followed by a hidden path π, followed by the states States and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Output: The conditional probability Pr(x|π) that x will be emitted given that the HMM follows the hidden path π.

Note: You may assume that transitions from the initial state occur with equal probability.

In [5]:
def poghpp(text):
    text = text.split('--------')
    string = text[0][:-1]
    matrix = pd.DataFrame(index=text[3][1:-1].split(' '),
                          columns=text[1][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[4])]).reshape((2,3)))
    hidden_path = text[2][1:-1]
    probability = 1
    for i in range(len(string)):
        probability*=matrix[string[i]][hidden_path[i]]
        
    return probability

In [6]:
text = '''zzzyxyyzzx
--------
x y z
--------
BAAAAAAAAA
--------
A B
--------
	x	y	z
A	0.176	0.596	0.228
B	0.225	0.572	0.203'''

In [7]:
poghpp(text)

3.5974895474624624e-06

## 3. Implement the Viterbi algorithm solving the Decoding Problem.

Input: A string x, followed by the alphabet from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Output: A path that maximizes the (unconditional) probability Pr(x, π) over all possible paths π.

Note: You may assume that transitions from the initial state occur with equal probability.

In [8]:
def viterbi(text):
    text = text.split('--------')
    string = text[0][:-1]
    transition = pd.DataFrame(index=text[2][1:-1].split(' '),
                          columns=text[2][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[3])]).reshape((2,2)))
    emission = pd.DataFrame(index=text[2][1:-1].split(' '),
                          columns=text[1][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[4])]).reshape((2,3)))
    path = []
    current = None
    for i in range(len(string)):
        if i==0:
            current = emission[string[0]].idxmax()
            path.append(current)
        else:
            current = (transition[current]*emission[string[i]]).idxmax()
            path.append(current)
    return path

In [9]:
text = '''xyxzzxyxyy
--------
x y z
--------
A B
--------
	A	B
A	0.641	0.359
B	0.729	0.271
--------
	x	y	z
A	0.117	0.691	0.192	
B	0.097	0.42	0.483'''

In [10]:
s=viterbi(text)

In [11]:
''.join(s)

'AAABBAAAAA'

## 4. Solve the Outcome Likelihood Problem.

Input: A string x, followed by the alphabet from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Output: The probability Pr(x) that the HMM emits x.

Note: You may assume that transitions from the initial state occur with equal probability.

In [15]:
def outcome_probability(text):
    text = text.split('--------')
    string = text[0][:-1]
    states = text[2][1:-1].split(' ')
    transition = pd.DataFrame(index=text[2][1:-1].split(' '),
                          columns=text[2][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[3])]).reshape((2,2)))
    emission = pd.DataFrame(index=text[2][1:-1].split(' '),
                          columns=text[1][1:-1].split(' '),
                          data=np.array([float(x) for x in re.findall("\d+\.\d+", text[4])]).reshape((2,3)))
    dic = {}
    dic[0] = {}
    for i in states:
        dic[0][i] = {}
        dic[0][i] = 1.0/len(states)*emission[string[0]][i]
        
    for i in range(1,len(string)):
        dic[i] = {}
        for j in states:
            dic[i][j] = 0
            for k in states:
                dic[i][j]+=dic[i-1][k]*transition[j][k]*emission[string[i]][j]
    res = 0
    for i in states:
        res+=dic[len(string)-1][i]
    return res

In [16]:
text = '''xzyyzzyzyy
--------
x y z
--------
A B
--------
	A	B
A	0.303	0.697 
B	0.831	0.169 
--------
	x	y	z
A	0.533	0.065	0.402 
B	0.342	0.334	0.324'''

In [17]:
outcome_probability(text)

1.1005510319694851e-06